# UDA-Hub: Multi-Agent Customer Support System

**Architecture**: Hierarchical (Supervisor) with 4 agents

This notebook demonstrates:
1. End-to-end workflow with structured logging
2. Tool usage (db_tools) during ticket processing
3. Confidence-based escalation from real similarity scores
4. Session memory inspection by thread_id
5. Long-term memory persistence
6. Edge-case handling (low/no KB match → escalation)

## 1. Setup

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import logging
logging.getLogger().setLevel(logging.INFO)

print("✅ Environment loaded")

In [ ]:
from agentic.workflow import orchestrator, get_session_state, CONFIDENCE_THRESHOLD
from langchain_core.messages import HumanMessage

print("✅ Orchestrator loaded")
print(f"✅ Confidence threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"\nWorkflow nodes: {list(orchestrator.get_graph().nodes.keys())}")

In [ ]:
def ask(message: str, thread_id: str = "demo", ticket_id: str = "") -> dict:
    """Send a message and get structured response."""
    result = orchestrator.invoke(
        input={
            "messages": [HumanMessage(content=message)],
            "ticket_id": ticket_id or thread_id,
            "tools_invoked": [],
        },
        config={"configurable": {"thread_id": thread_id}}
    )
    return {
        "response": result["messages"][-1].content,
        "category": result.get("category", "unknown"),
        "escalated": result.get("needs_escalation", False),
        "confidence": result.get("knowledge_confidence", 0.0),
        "tools_used": result.get("tools_invoked", [])
    }

print("✅ Helper ready")

---
## 2. Tool Usage Demo (db_tools)

This demonstrates `get_user_info` and `get_user_reservations` being invoked during workflow.

**Watch the structured logs** showing: tool_invoked → tool_result

In [ ]:
# First, let's see what users exist in the database
from agentic.tools.db_tools import get_user_info, get_user_reservations, check_subscription_status

print("=" * 60)
print("DIRECT TOOL INVOCATION TEST")
print("=" * 60)

# Test get_user_info tool
print("\n📧 Testing get_user_info('alice@example.com'):")
print("-" * 40)
result = get_user_info.invoke("alice@example.com")
print(result)

In [ ]:
# Test get_user_reservations tool
print("\n📅 Testing get_user_reservations('alice@example.com'):")
print("-" * 40)
result = get_user_reservations.invoke("alice@example.com")
print(result)

In [ ]:
# Now test tools being invoked DURING workflow
print("\n" + "=" * 60)
print("TOOL USAGE DURING WORKFLOW")
print("=" * 60)
print("\n🔍 Watch the JSON logs below showing tool invocations:\n")

result = ask(
    "Hi, I'm alice@example.com and I need help with my subscription",
    thread_id="tool-demo-1"
)

print("\n" + "=" * 60)
print("RESULT SUMMARY")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")
print(f"Tools Used: {result['tools_used']}")
print(f"\nResponse: {result['response'][:300]}...")

---
## 3. RAG Distance Analysis

First, let's see the ACTUAL distances returned by Chroma to understand the confidence scoring.

In [ ]:
# Direct access to Chroma to see raw distances
from agentic.tools.rag_tools import get_vector_store

vector_store = get_vector_store()

test_queries = [
    "How do I reset my password?",                    # Should be LOW distance (good match)
    "What's included in the premium tier?",           # Should be LOW distance
    "I want a refund",                                # Should be LOW distance
    "Enterprise API webhook integration B2B",         # Should be HIGH distance (bad match)
    "Quantum computing blockchain cryptocurrency",    # Should be HIGH distance
    "asdfghjkl random gibberish xyz123",              # Should be VERY HIGH distance
]

print("RAW CHROMA DISTANCES (lower = better match)")
print("=" * 70)
print(f"\n{'Query':<45} {'Best':<8} {'Avg':<8} {'Worst'}")
print("-" * 70)

for query in test_queries:
    results = vector_store.similarity_search_with_score(query, k=3)
    distances = [score for _, score in results]
    best = min(distances)
    avg = sum(distances) / len(distances)
    worst = max(distances)
    print(f"{query[:43]:<45} {best:<8.3f} {avg:<8.3f} {worst:.3f}")

print("\n" + "=" * 70)
print("Distance interpretation:")
print("  < 0.3 = Highly relevant (confidence 95%)")
print("  0.3-0.5 = Relevant (confidence 70-80%)")
print("  0.5-0.6 = Weak match (confidence 55% - triggers escalation)")
print("  > 0.6 = Poor match (confidence 40% or less)")

---
## 4. Edge-Case Demo: Low Confidence → Escalation

Based on the distances above, let's test queries that SHOULD trigger escalation.

In [ ]:
from agentic.tools.rag_tools import search_knowledge_with_scores

print("CONFIDENCE SCORING TEST")
print("=" * 70)
print(f"Threshold for escalation: < {CONFIDENCE_THRESHOLD:.0%}")
print(f"\n{'Query':<45} {'Confidence':<12} {'Action'}")
print("-" * 70)

for query in test_queries:
    content, confidence = search_knowledge_with_scores(query)
    action = "✅ Resolve" if confidence >= CONFIDENCE_THRESHOLD else "⚠️ ESCALATE"
    print(f"{query[:43]:<45} {confidence:.0%}{'':>8} {action}")

In [ ]:
# Test a COMMON query - should RESOLVE
print("=" * 60)
print("TEST 1: Common Query (Should RESOLVE)")
print("=" * 60)
print("Query: 'How do I reset my password?'\n")

result_common = ask("How do I reset my password?", thread_id="edge-case-1")

print(f"\n✅ Confidence: {result_common['confidence']:.0%}")
print(f"✅ Threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"✅ Escalated: {result_common['escalated']}")
print(f"✅ Category: {result_common['category']}")

In [ ]:
# Test a GIBBERISH query - should ESCALATE
print("\n" + "=" * 60)
print("TEST 2: Gibberish Query (Should ESCALATE)")
print("=" * 60)
print("Query: 'asdfghjkl random gibberish xyz123'\n")

result_gibberish = ask(
    "asdfghjkl random gibberish xyz123",
    thread_id="edge-case-gibberish"
)

print(f"\n⚠️ Confidence: {result_gibberish['confidence']:.0%}")
print(f"⚠️ Threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"⚠️ Escalated: {result_gibberish['escalated']}")

if result_gibberish['escalated']:
    print("\n✅ SUCCESS: Low confidence query correctly triggered escalation!")
else:
    print("\n❌ Query did not escalate")

In [ ]:
# Test an OBSCURE technical query - should ESCALATE
print("\n" + "=" * 60)
print("TEST 3: Obscure Technical Query (Should ESCALATE)")
print("=" * 60)
print("Query: 'Quantum computing blockchain cryptocurrency partnership'\n")

result_obscure = ask(
    "Quantum computing blockchain cryptocurrency partnership",
    thread_id="edge-case-obscure"
)

print(f"\n⚠️ Confidence: {result_obscure['confidence']:.0%}")
print(f"⚠️ Threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"⚠️ Escalated: {result_obscure['escalated']}")

if result_obscure['escalated']:
    print("\n✅ SUCCESS: Low confidence query correctly triggered escalation!")
else:
    print("\n❌ Query did not escalate")

In [ ]:
# Comparison table
print("\n" + "=" * 70)
print("COMPARISON: Resolution vs Escalation Paths")
print("=" * 70)
print(f"\n{'Query Type':<25} {'Confidence':<12} {'Escalated':<12} {'Path'}")
print("-" * 70)
print(f"{'Password reset':<25} {result_common['confidence']:.0%}{'':>8} {str(result_common['escalated']):<12} {'Resolve' if not result_common['escalated'] else 'Escalate'}")
print(f"{'Gibberish':<25} {result_gibberish['confidence']:.0%}{'':>8} {str(result_gibberish['escalated']):<12} {'Resolve' if not result_gibberish['escalated'] else 'Escalate'}")
print(f"{'Quantum/blockchain':<25} {result_obscure['confidence']:.0%}{'':>8} {str(result_obscure['escalated']):<12} {'Resolve' if not result_obscure['escalated'] else 'Escalate'}")
print(f"\nThreshold: {CONFIDENCE_THRESHOLD:.0%}")

---
## 5. Session Memory Inspection

Demonstrates inspecting stored session state by `thread_id`.

In [ ]:
import uuid

# Create a new session with multiple messages
session_id = f"session-inspect-{uuid.uuid4().hex[:8]}"
print(f"Session ID: {session_id}")
print("=" * 60)

# First message
print("\n[Message 1] User: I'm having trouble with my subscription")
result1 = ask("I'm having trouble with my subscription", thread_id=session_id)
print(f"Agent: {result1['response'][:150]}...")

In [ ]:
# Second message in same session
print(f"\n[Message 2] User: What benefits do I get?")
result2 = ask("What benefits do I get?", thread_id=session_id)
print(f"Agent: {result2['response'][:150]}...")

In [ ]:
# Now inspect the session state
print("\n" + "=" * 60)
print("SESSION STATE INSPECTION")
print("=" * 60)
print(f"\nRetrieving state for thread_id: {session_id}\n")

state = get_session_state(session_id)

if state:
    print("📋 Stored Session State:")
    print("-" * 40)
    print(f"Category: {state['category']}")
    print(f"Needs Escalation: {state['needs_escalation']}")
    print(f"Confidence: {state['knowledge_confidence']:.0%}" if state['knowledge_confidence'] else "N/A")
    print(f"Tools Invoked: {state['tools_invoked']}")
    print(f"\n📨 Messages in Session ({len(state['messages'])}):") 
    for i, msg in enumerate(state['messages'], 1):
        role = msg['role'].upper()
        content = msg['content'][:80] + "..." if len(msg['content']) > 80 else msg['content']
        print(f"  {i}. [{role}]: {content}")
else:
    print("❌ No state found for this session")

---
## 6. Long-Term Memory Demo

Demonstrates cross-session memory persistence using `ticket_id`.

In [ ]:
from agentic.tools.memory import get_history, store_interaction, format_history_for_context

# Create a ticket for persistent memory
ticket_id = f"ticket-{uuid.uuid4().hex[:8]}"
print(f"Ticket ID: {ticket_id}")
print("=" * 60)

# Session 1
print("\n[SESSION 1]")
result1 = ask(
    "I need to cancel my reservation for the art gallery",
    thread_id=f"sess1-{ticket_id}",
    ticket_id=ticket_id
)
print(f"User: I need to cancel my reservation for the art gallery")
print(f"Agent: {result1['response'][:200]}...")

In [ ]:
# Check what's stored in long-term memory
print("\n📦 Long-Term Memory (TicketMessage table):")
print("-" * 40)

history = get_history(ticket_id, limit=10)
print(f"Messages stored: {len(history)}")
for msg in history:
    print(f"  [{msg['role'].upper()}]: {msg['content'][:60]}...")

In [ ]:
# Session 2 - NEW session, SAME ticket (memory should persist)
print("\n[SESSION 2 - New session, same ticket]")
result2 = ask(
    "Actually, can you remind me what we discussed?",
    thread_id=f"sess2-{ticket_id}",  # Different session
    ticket_id=ticket_id  # Same ticket = access to history
)
print(f"User: Actually, can you remind me what we discussed?")
print(f"Agent: {result2['response'][:300]}...")

---
## 7. End-to-End Workflow with Full Logging

Complete run showing: classification → routing → KB retrieval → tool usage → resolution/escalation

In [ ]:
print("=" * 60)
print("END-TO-END RUN: Resolution Path")
print("=" * 60)
print("\n📝 Query: 'How do I book an experience with my premium subscription?'\n")
print("Watch the structured JSON logs below:\n")

result = ask(
    "How do I book an experience with my premium subscription?",
    thread_id="e2e-resolve"
)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")
print(f"Tools: {result['tools_used']}")

In [ ]:
print("\n" + "=" * 60)
print("END-TO-END RUN: Escalation Path (Blocked Account)")
print("=" * 60)
print("\n📝 Query: 'My account has been blocked!'\n")
print("Watch the structured JSON logs below:\n")

result = ask(
    "My account has been blocked!",
    thread_id="e2e-escalate"
)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")

In [ ]:
print("\n" + "=" * 60)
print("END-TO-END RUN: Escalation Path (Low Confidence)")
print("=" * 60)
print("\n📝 Query: 'I need help with quantum blockchain integration'\n")
print("Watch the logs - should show low_confidence_escalation event:\n")

result = ask(
    "I need help with quantum blockchain integration",
    thread_id="e2e-low-conf"
)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")

if result['escalated'] and result['confidence'] < CONFIDENCE_THRESHOLD:
    print("\n✅ SUCCESS: Low confidence correctly triggered escalation!")

---
## Summary

### Demonstrated:
1. ✅ **Tool Usage**: `get_user_info`, `get_user_reservations` invoked during workflow with logging
2. ✅ **Edge-Case**: Low confidence KB match → automatic escalation  
3. ✅ **Session Memory**: Inspection of stored state by `thread_id`
4. ✅ **Long-Term Memory**: Cross-session persistence via `ticket_id`
5. ✅ **Structured Logging**: JSON format with node/event/data fields
6. ✅ **Real Confidence Scores**: From Chroma distances (not string heuristics)